In [15]:
import pandas as pd
from data_tweak.converters import convert_datetime, convert_race, convert_sex

stops_df = pd.read_csv("../files/events/stops_full.csv")
stops_df.dropna(subset=["date_time"], how="any", inplace=True)
convert_datetime(stops_df)
# this needs to be sorted so we can slice properly
stops_df.set_index("date_time", inplace=True)
stops_df.sort_index(inplace=True)
convert_race(stops_df, ["first_off_race", "second_off_race", "civ_race"])
convert_sex(stops_df, ["first_off_sex", "second_off_sex", "civ_sex"])


/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_58378/4243400398.py:4: DtypeWarning: Columns (3,5,6,7,8,10,11,12,13,20,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  stops_df = pd.read_csv("../files/events/stops_full.csv")


In [56]:

import numpy as np

"""
  @TODO bring in arrest/uof data
  @TODO be more explicit about data we're excluding; record it
  @TODO identify potential mis-merges
  @TODO add total_stops field
  @TODO output some metadata about the merge for QA
  @TODO fix output file formatting (header row being appended to file for each chunk)
  @TODO add correct timezones to data
"""

with pd.read_csv("../files/events/assignment_cleaned_first_200k.csv", chunksize=10000) as reader:
  left_on = ["off_last_name", "off_first_name", "off_race", "off_sex"]
  right_on_first_off = ["first_off_last_name", "first_off_first_name", "first_off_race", "first_off_sex"]
  right_on_second_off = ["second_off_last_name", "second_off_first_name", "second_off_race", "second_off_sex"]
  group_by = ["shift_start", "shift_end_corrected", "is_working", "off_first_name", "off_last_name", "off_sex", "off_race", "watch", "beat_x"]
  for chunk in reader:
    chunk.dropna(subset=["shift_start", "shift_end_corrected"], how="any", inplace=True)
    convert_datetime(chunk, datetime_col="shift_start")
    convert_datetime(chunk, datetime_col="shift_end_corrected")
    convert_race(chunk, ["off_race"])
    convert_sex(chunk, ["off_sex"])
    chunk.set_index("shift_start", inplace=True, drop=False)
    window_start = chunk["shift_start"].min()
    window_end = chunk["shift_end_corrected"].max()
    stops_df_window = stops_df[window_start:window_end]
    stops_merged = pd.merge(chunk, stops_df_window, how="outer", left_on=left_on, right_on=right_on_first_off)
    stops_merged.assign(first_officer=True)
    stops_df_window = stops_df_window.dropna(subset=right_on_second_off, how="any")
    second_off_merged = pd.merge(chunk, stops_df_window, how="outer", left_on=left_on, right_on=right_on_second_off)
    second_off_merged.assign(first_officer=False)
    # second_off_merged.dropna(subset=right_on_second_off).to_csv("../files/events/assignment_merged/second_off_merge_test.csv", mode="a")
    stops_merged = pd.concat([stops_merged, second_off_merged])
    stops_merged = stops_merged.assign(
      stops_white = np.where(stops_merged["civ_race"] == "WHITE", 1, 0),
      stops_black = np.where(stops_merged["civ_race"] == "BLACK", 1, 0),
      stops_hispanic = np.where(stops_merged["civ_race"] == "HISPANIC", 1, 0),
      stops_apa = np.where(stops_merged["civ_race"] == "ASIAN/PACIFIC ISLANDER", 1, 0),
      stops_native = np.where(stops_merged["civ_race"] == "NATIVE AMERICAN/ALASKAN NATIVE", 1, 0),
      stops_male = np.where(stops_merged["civ_sex"] == "MALE", 1, 0),
      stops_female = np.where(stops_merged["civ_sex"] == "FEMALE", 1, 0)
    )
    stops_merged[stops_merged[left_on].isna().all(axis=1)].to_csv("../files/events/assignment_merged/merge_no_match_test.csv", mode="a")
    stops_merged.dropna(subset=left_on, how="all", inplace=True)
    grped = stops_merged.groupby(group_by).agg({"stops_white":sum, "stops_black":sum, "stops_hispanic":sum, "stops_apa":sum, "stops_native":sum, "stops_male":sum, "stops_female":sum})
    grped = grped.assign(
      stops_total = grped["stops_white"] + grped["stops_black"] + grped["stops_hispanic"] + grped["stops_apa"] + grped["stops_native"]
    )
    grped.to_csv("../files/events/assignment_merged/merge_grp_test.csv", mode="a")
    is_working_stats = grped.reset_index().groupby("is_working").agg({"stops_total": sum}).reset_index()
    print(is_working_stats)


/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_58378/3985243628.py:28: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  stops_df_window = stops_df[window_start:window_end]


  is_working  stops_total
0          N          237
1          Y         2077


/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_58378/3985243628.py:28: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  stops_df_window = stops_df[window_start:window_end]


  is_working  stops_total
0          N         1131
1          Y         3992


/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_58378/3985243628.py:28: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  stops_df_window = stops_df[window_start:window_end]


...

  is_working  stops_total
0          N         1416
1          Y         5614


/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_58378/3985243628.py:28: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  stops_df_window = stops_df[window_start:window_end]


  is_working  stops_total
0          N         1249
1          Y         4923


/var/folders/9h/xh9hskwj4q9bvjqcy131pv9c0000gn/T/ipykernel_58378/3985243628.py:28: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  stops_df_window = stops_df[window_start:window_end]


  is_working  stops_total
0          N         1225
1          Y         4409
